<a href="https://colab.research.google.com/github/hyona-yu/HAR_python/blob/master/har_with_uci_data_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import pandas as pd
import random

In [2]:
from sklearn.metrics import classification_report

In [3]:
device ='cuda' if torch.cuda.is_available() else 'cpu'
random.seed(42)
torch.manual_seed(42)
if device =='cuda':
    torch.cuda.manual_seed_all(42)

In [4]:
! git clone https://github.com/hyona-yu/Dataset.git

Cloning into 'Dataset'...
remote: Enumerating objects: 660, done.
remote: Total 660 (delta 0), reused 0 (delta 0), pack-reused 660
Receiving objects: 100% (660/660), 427.85 MiB | 23.97 MiB/s, done.
Resolving deltas: 100% (140/140), done.
Checking out files: 100% (626/626), done.


In [5]:
url = 'Dataset/UCI HAR Dataset/'

In [6]:
#time step 128
signal = ['body_acc_x_', 'body_acc_y_','body_acc_z_','body_gyro_x_','body_gyro_y_','body_gyro_z_','total_acc_x_','total_acc_y_','total_acc_z_']

In [7]:
train_csv = pd.DataFrame()
for s in signal:
  train_path = str(url + "train/Inertial Signals/"+ s + "train.txt")
  train_csv = pd.concat([train_csv, pd.read_csv(train_path, sep ='\s+', header = None)], axis = 1)

In [8]:
train_csv

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
0,0.000181,0.010139,0.009276,0.005066,0.010810,0.004045,0.004757,0.006214,0.003307,0.007572,0.005407,0.006221,0.006895,0.004610,0.007331,0.005078,0.005763,0.006693,0.005443,0.008241,0.006506,0.006532,0.007422,0.005772,0.006240,0.004623,0.004833,0.005499,0.004341,0.005098,0.004269,0.003045,0.003204,0.004520,0.005127,0.003550,0.004234,0.004469,0.003573,0.005136,...,0.101472,0.101226,0.102653,0.101608,0.100634,0.096142,0.090326,0.089905,0.093117,0.095741,0.094250,0.094481,0.097755,0.099002,0.098523,0.095429,0.094212,0.097735,0.102188,0.103457,0.099610,0.097589,0.098616,0.097882,0.096877,0.097010,0.097582,0.097073,0.097425,0.099341,0.100058,0.098564,0.093177,0.088742,0.090505,0.094843,0.098350,0.100385,0.099874,0.094987
1,0.001094,0.004550,0.002879,0.002247,0.003305,0.002416,0.001619,0.000981,0.000009,-0.000363,-0.000487,-0.000356,-0.000229,-0.000131,-0.000441,-0.001565,-0.000929,0.000009,-0.001236,-0.000911,-0.000435,-0.001177,0.000165,0.000751,0.000389,0.001328,0.002053,0.002738,0.002743,0.001330,-0.000367,-0.000722,-0.001904,-0.004294,-0.000025,0.005251,0.002490,0.000811,0.001166,-0.000693,...,0.097527,0.097369,0.098893,0.100536,0.098083,0.097308,0.101762,0.105788,0.104529,0.101473,0.096284,0.087116,0.088206,0.096796,0.096812,0.098964,0.104573,0.103425,0.100734,0.097965,0.099716,0.104411,0.103337,0.103670,0.103047,0.099247,0.100684,0.100973,0.097971,0.095684,0.094537,0.098759,0.101977,0.095360,0.089466,0.095126,0.099496,0.093535,0.089035,0.090612
2,0.003531,0.002285,-0.000420,-0.003738,-0.006706,-0.003148,0.000733,0.000668,0.002162,-0.000946,-0.006476,-0.003423,-0.000610,-0.002929,-0.001796,0.000956,0.002311,0.002538,0.001180,0.001217,0.000869,0.000686,0.002156,0.000478,-0.001506,-0.002105,-0.001494,0.002181,0.002876,0.000871,-0.000392,-0.000569,0.000569,-0.001176,-0.001957,-0.000471,0.000208,0.003380,0.004829,0.003380,...,0.093599,0.089887,0.083149,0.085446,0.089210,0.092701,0.095387,0.093006,0.089667,0.085096,0.084867,0.090206,0.091938,0.094822,0.096023,0.093216,0.096160,0.096263,0.093678,0.096380,0.093724,0.087680,0.085679,0.086568,0.090691,0.092891,0.093875,0.094094,0.087741,0.082867,0.085343,0.087155,0.084546,0.082166,0.081972,0.081413,0.081936,0.083011,0.082334,0.081487
3,-0.001772,-0.001311,0.000388,0.000408,-0.000355,0.000998,0.001109,-0.003149,-0.008882,-0.010483,-0.004482,0.004528,0.008167,0.002929,-0.004487,-0.004717,-0.001637,-0.000097,0.001614,0.002619,0.004765,0.005851,0.002579,0.000677,0.002138,0.003519,0.002715,0.002370,0.003299,0.002357,0.001481,-0.000340,-0.002201,0.000124,0.002345,0.002706,0.001281,-0.000949,0.000890,0.004952,...,0.082965,0.081726,0.085902,0.088833,0.086746,0.084692,0.082640,0.084150,0.085247,0.083578,0.083925,0.087735,0.092079,0.090971,0.088221,0.087130,0.084942,0.082944,0.079419,0.076888,0.080105,0.084625,0.087186,0.087998,0.087082,0.085528,0.085230,0.088954,0.093179,0.091213,0.088112,0.087322,0.083738,0.082701,0.084490,0.082785,0.084084,0.085761,0.083275,0.081404
4,0.000087,-0.000272,0.001022,0.003126,0.002284,0.000885,0.001933,0.002270,0.002247,0.002175,0.001750,0.001420,0.000297,0.000665,0.001911,0.001778,0.001087,-0.000716,-0.001347,-0.000384,-0.000135,0.000666,0.000656,0.000221,0.001470,0.001644,-0.000106,-0.001683,-0.001165,0.000241,0.000216,-0.000102,-0.001546,-0.003518,-0.003204,-0.002376,-0.001825,-0.001754,-0.002504,-0.002612,...,0.084108,0.083109,0.080163,0.076622,0.078279,0.084254,0.085648,0.081490,0.084675,0.088211,0.086471,0.087503,0.082953,0.079125,0.084738,0.087178,0.085370,0.084911,0.090154,0.093388,0.083022,0.074595,0.079912,0.085177,0.082895,0.084075,0.085648,0.082100,0.086196,0.089715,0.088028,0.090312,0.088713,0.086957,0.086522,0.081640,0.079652,0.081329,0.085397,0.088816
...,...,...,...,...,...,...,...,...,...,

In [9]:
test_csv = pd.DataFrame()
for s in signal:
  test_path = str(url + "test/Inertial Signals/"+ s + "test.txt")
  test_csv = pd.concat([test_csv, pd.read_csv(test_path, sep ='\s+', header = None)], axis = 1)

In [10]:
test_csv

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
0,0.011653,0.013109,0.011269,0.027831,0.002318,-0.018965,-0.061920,-0.094248,-0.079534,-0.070084,-0.019152,-0.002267,-0.030111,-0.011910,-0.015542,-0.016666,-0.006630,-0.023481,-0.017628,-0.021150,-0.032659,-0.022478,-0.010197,-0.003120,-0.016496,-0.020442,-0.027568,-0.055032,-0.057480,-0.047096,-0.028362,-0.017185,-0.007376,0.024779,0.023929,0.021248,0.009700,-0.035119,-0.010733,0.023672,...,0.160443,0.151523,0.147972,0.154167,0.153757,0.151731,0.146116,0.143020,0.143608,0.144273,0.149575,0.150313,0.149967,0.150430,0.148942,0.149977,0.147962,0.148390,0.150847,0.150729,0.152542,0.151450,0.153751,0.158655,0.158038,0.158718,0.158712,0.159610,0.161821,0.158132,0.155110,0.153346,0.149289,0.147401,0.146905,0.145261,0.143904,0.144395,0.144703,0.145494
1,0.009280,0.004930,0.003954,0.009214,0.016156,0.014079,0.013105,0.021247,0.026823,0.031054,0.037886,0.026463,-0.021441,-0.060568,-0.024725,0.020272,-0.001934,-0.008907,0.009538,-0.004332,-0.001864,0.004955,-0.005336,0.002001,-0.001708,-0.006463,0.000204,-0.003640,0.000083,-0.001442,-0.005959,0.000344,0.000558,0.002381,0.001828,-0.001456,-0.000200,-0.000570,0.002605,0.002644,...,0.124839,0.125669,0.126854,0.133731,0.140218,0.134717,0.125654,0.119328,0.119537,0.122430,0.120289,0.121756,0.128699,0.132581,0.131261,0.129647,0.133942,0.137049,0.135974,0.138160,0.138747,0.134309,0.127496,0.122937,0.125959,0.129914,0.131720,0.133536,0.134832,0.135538,0.133731,0.132851,0.132054,0.132224,0.139767,0.147877,0.153025,0.152788,0.139843,0.121314
2,0.005732,0.007066,0.005110,0.002434,0.002024,0.001382,0.001930,0.004875,0.003776,0.002400,0.002535,-0.000113,0.000473,0.002405,0.001630,0.002221,0.002135,0.002463,0.003525,0.001243,0.000013,0.002337,0.004713,0.004801,0.003798,0.003253,0.000918,-0.000773,0.000259,0.000188,-0.000269,0.000050,0.000772,0.004269,0.007619,0.004565,0.000272,0.000717,-0.000957,-0.003173,...,0.141447,0.138268,0.136213,0.135355,0.136989,0.139679,0.143010,0.146465,0.144218,0.143410,0.143945,0.142914,0.146899,0.150695,0.150838,0.147119,0.142718,0.136946,0.129181,0.134041,0.139226,0.134179,0.133947,0.131320,0.125988,0.125503,0.124250,0.124186,0.124420,0.126557,0.131392,0.132404,0.134938,0.137589,0.134431,0.133174,0.133712,0.132678,0.132694,0.132117
3,0.000452,0.000604,-0.002485,-0.004562,-0.006002,-0.006382,-0.005276,-0.001489,0.005153,0.004951,0.000732,0.000563,-0.003597,-0.007371,-0.006113,-0.007007,-0.006515,-0.005303,-0.005497,-0.004017,-0.001901,0.000642,-0.002360,-0.007317,-0.003625,-0.000644,-0.001341,-0.000848,-0.003763,-0.005992,-0.004094,-0.000786,0.003458,0.003703,-0.000745,-0.003801,-0.002695,-0.000235,-0.000247,-0.000706,...,0.116925,0.124498,0.131239,0.124867,0.123530,0.126676,0.126094,0.128428,0.128808,0.129597,0.133210,0.131184,0.128958,0.129109,0.129185,0.133316,0.137717,0.137781,0.132310,0.121513,0.120037,0.132227,0.139081,0.135362,0.129449,0.126648,0.127680,0.128090,0.127752,0.127270,0.126040,0.124815,0.125353,0.126463,0.121757,0.117611,0.118865,0.116060,0.110997,0.111912
4,-0.004362,-0.002765,-0.004905,-0.004682,-0.000267,0.004135,0.005626,0.000933,-0.001542,0.003020,0.007453,0.008221,0.004644,0.000727,-0.001039,-0.004051,-0.006519,-0.004500,-0.001682,-0.002965,-0.002603,0.000176,-0.001433,-0.003978,-0.005538,-0.005916,-0.003250,-0.001155,0.000648,0.001719,-0.000121,-0.003209,-0.005932,-0.005502,-0.003120,-0.001452,-0.000775,-0.001839,-0.003113,-0.003741,...,0.112746,0.115978,0.115545,0.111981,0.114324,0.112818,0.111041,0.113719,0.112479,0.115182,0.123094,0.127987,0.126593,0.120257,0.117611,0.118280,0.119715,0.121267,0.117990,0.116020,0.119529,0.121717,0.119389,0.114924,0.113130,0.113738,0.114183,0.115353,0.121068,0.129294,0.129091,0.125772,0.126042,0.121086,0.118511,0.125458,0.129365,0.125895,0.122510,0.122

In [11]:
train_y = pd.read_csv(url + 'train/y_train.txt', header = None, names= ['label'])
test_y =pd.read_csv(url + 'test/y_test.txt', header =None, names= ['label'])

In [12]:
for i in range(len(train_y)):
  if train_y.loc[i,'label'] >=7 :
    train_csv.drop(i, inplace =True)
    train_y.drop(i, inplace = True)

In [13]:
for i in range(len(test_y)):
  if test_y.loc[i,'label'] >=7 :
    test_csv.drop(i, inplace =True)
    test_y.drop(i, inplace = True)

In [14]:
train_D = np.array(train_csv).reshape(train_csv.shape[0], 128, -1)

In [15]:
train_D.shape

(7352, 128, 9)

In [16]:
test_D = np.array(test_csv).reshape(test_csv.shape[0],128,-1)

In [17]:
test_D = torch.FloatTensor(test_D)

In [19]:

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
encoder.fit(train_y)
train_L = encoder.transform(train_y).toarray()

In [20]:
test_L = encoder.transform(test_y).toarray()

In [21]:
train_D = torch.FloatTensor(train_D)
train_L = torch.FloatTensor(np.array(train_L))

In [22]:
import torch.utils
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_D, train_L), batch_size = 100, shuffle= False, drop_last = True)

#SVM

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(random_state = 42, decision_function_shape='ova')
params = {
    'C' : [0.5,1,10]
}
cv = GridSearchCV(clf, param_grid = params)
cv.fit(train_csv, train_y)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ova', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=42, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None, param_grid={'C': [0.5, 1, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
pred = cv.predict(test_csv)

acc = accuracy_score(pred, test_y)
print("SVM :",acc)

SVM : 0.8982015609093994


In [ ]:
print(classification_report(pred, test_y)) #support는 정밀도의 평균? 

              precision    recall  f1-score   support

           1       0.96      0.91      0.94       520
           2       0.92      0.90      0.91       480
           3       0.89      0.90      0.89       418
           4       0.77      0.84      0.80       452
           5       0.84      0.83      0.84       540
           6       1.00      1.00      1.00       537

    accuracy                           0.90      2947
   macro avg       0.90      0.90      0.90      2947
weighted avg       0.90      0.90      0.90      2947



#CNN

In [ ]:
class CNN(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(CNN, self).__init__()
# filter 수 늘리는게 중요? kernel size 크게,

        self.layer1 = torch.nn.Sequential(torch.nn.Conv1d(input_size ,128, kernel_size= 5, stride= 1), torch.nn.MaxPool1d(kernel_size= 1), 
                                          torch.nn.ReLU()) #
        self.layer2 = torch.nn.Sequential(torch.nn.Conv1d(128,64, kernel_size= 3, stride= 1, padding =0),
                                          torch.nn.MaxPool1d(kernel_size= 1, stride = 1, padding = 0), 
                                          torch.nn.ReLU())
        self.f1 = torch.nn.Flatten()
        self.layer3 = torch.nn.Linear(192 , output_size, bias = True) #(input_size - kernel_size + 2*padding_size)/stride + 1
        #self.tanh = torch.nn.Tanh()
        torch.nn.init.kaiming_uniform(self.layer3.weight)

    def forward(self, x):
        out = self.layer1(x)
        #print(out.shape)
        out = self.layer2(out)
        #print(out.shape)
        out = self.layer3(out)
        #print(out.shape)
        out = out.view(out.size(0), -1)
        #print(out.shape)
        out = self.layer3(out)
        return out


In [ ]:
model = CNN(train_D.shape[1], train_L.shape[1]).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  


In [ ]:
running_loss =0
for e in range(1001):
  for i , data in enumerate(train_loader):
    x, y = data
    #print(x.shape)
    x = x.to(device)
    y = y.to(device)
    optimizer.zero_grad()
    h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  if (e+1) % 100 == 0:
    print('[%d] loss : %.3f' %(e, running_loss / 100))
    running_loss =0

print('fin')

RuntimeError: ignored

In [ ]:
with torch.no_grad():
  model.eval()
  pred=  model(test_D.to(device))


In [ ]:
train_L

In [ ]:
test_y.shape

In [ ]:
np.array(pred).shape

In [ ]:
acc = [np.argmax(pred_lstm.cpu()[i])== np.argmax(test_L[i]) for i in range(len(test_D))]
acc = np.array(acc, dtype=np.float32)
real_acc = acc.mean()
print('CNN : ', real_acc)

CNN :  0.7960638


In [ ]:
print(classification_report(np.round(pred.cpu()), np.array(test_y)))

#LSTM

In [ ]:
train_D[0][0]

In [ ]:
#step size : 128
#input = (batch_size, time_steps, input_size)
#rnn( input_size, hidden_size)

In [ ]:
class LSTM(torch.nn.Module):
  def __init__(self, n_class, bi):
    super(LSTM, self).__init__()

    self.lstm = torch.nn.LSTM(9, 32)
    #self.lstm2 = torch.nn.LSTM(32, 32)
    self.l1 = torch.nn.Linear(128*32, n_class)
    self.dropout = torch.nn.Dropout(0.1)
    self.tanh = torch.nn.Tanh()
    self.softmax = torch.nn.Softmax(dim = 1)

  def forward(self, x):
    #print(x.shape)
    #x = x.permute(1,0,2)# 축 바꾸기. 전치행렬처럼
    #print(x.shape)
    out, hidden = self.lstm(x)
    #out, _  = self.lstm2(out, hidden)
    #out, _ = self.lstm2(out, hidden)
    #print(out.shape)
    out = self.tanh(out)
    out = self.dropout(out)
    out = out.contiguous().view(-1, 128*32)
    #print(out.shape)
    out = self.l1(out)
    #out = self.softmax(out)
    return out
#소프트맥스 지말고 탄젠트나써라

In [ ]:
model = LSTM(train_L.shape[1], False).to(device)

In [ ]:
train_L.shape[1]

6

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

In [ ]:
running_loss =0
for e in range(201):
  for i , data in enumerate(train_loader):
    x, y = data
    x = x.to(device)
    y = y.to(device)
    optimizer.zero_grad()
    h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  if e % 10 == 0:
    print('[%d] loss : %.3f' %(e, running_loss / 100))
    running_loss =0

print('fin')

[0] loss : 0.078
[10] loss : 0.424
[20] loss : 0.231
[30] loss : 0.177
[40] loss : 0.163
[50] loss : 0.137
[60] loss : 0.129
[70] loss : 0.117
[80] loss : 0.113
[90] loss : 0.112
[100] loss : 0.103
[110] loss : 0.102
[120] loss : 0.097
[130] loss : 0.094
[140] loss : 0.093
[150] loss : 0.086
[160] loss : 0.086
[170] loss : 0.084
[180] loss : 0.080
[190] loss : 0.079
[200] loss : 0.078
fin


In [ ]:
test_D.shape

In [ ]:
with torch.no_grad():
  model.eval()
  pred_lstm = model(test_D.to(device))


In [ ]:
pred_lstm.shape

In [ ]:
train_L.shape

In [ ]:
acc = [np.argmax(pred_lstm.cpu()[i])== np.argmax(test_L[i]) for i in range(len(test_D))]
acc = np.array(acc, dtype=np.float32)
real_acc = acc.mean()
print('LSTM : ', real_acc)

LSTM :  0.9290804


In [ ]:
#print(classification_report(np.round(pred_lstm.cpu()), np.array(test_y)))

#CNN + SVM

In [ ]:
class CNN(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(CNN, self).__init__()
# filter 수 늘리는게 중요? kernel size 크게,

        self.layer1 = torch.nn.Sequential(torch.nn.Conv1d(input_size ,128, kernel_size= 5, stride= 1), torch.nn.MaxPool1d(kernel_size= 1), 
                                          torch.nn.ReLU()) #
        self.layer2 = torch.nn.Sequential(torch.nn.Conv1d(128,64, kernel_size= 3, stride= 1, padding =0),
                                          torch.nn.MaxPool1d(kernel_size= 1, stride = 1, padding = 0), 
                                          torch.nn.ReLU())
        self.f1 = torch.nn.Flatten()
        self.layer3 = torch.nn.Linear(192 , output_size, bias = True) #(input_size - kernel_size + 2*padding_size)/stride + 1
        #self.tanh = torch.nn.Tanh()
        self.softmax = torch.nn.Softmax()
        torch.nn.init.kaiming_uniform(self.layer3.weight)

    def forward(self, x):
        out = self.layer1(x)
        #print(out.shape)
        out = self.layer2(out)
        #print(out.shape)
        out = out.view(out.size(0), -1)
        out = self.f1(out)
        #print(out.shape)
        model = self.layer3(out)
        model = self.tanh(model)
        #model = self.tanh(model)
        return out, model

In [ ]:
model = CNN(train_D.shape[1], train_L.shape[1]).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  


In [ ]:
running_loss = 0
new_train = []
epoch =1001
for e in range(epoch + 1):
  for i , data in enumerate(train_loader):
    x, y = data
    #print(x.shape)
    x = x.to(device)
    y = y.to(device)
    optimizer.zero_grad()
    new_x, h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if e == epoch:
        new_train.extend(new_x.cpu().detach().numpy())

  if (e+1) % 100 == 0:
    print('[%d] loss : %.3f' %(e, running_loss / 100))
    running_loss =0
  

print('fin')

[99] loss : 0.273
[199] loss : 0.210
[299] loss : 0.180
[399] loss : 0.151
[499] loss : 0.121
[599] loss : 0.126
[699] loss : 0.103
[799] loss : 0.089
[899] loss : 0.087
[999] loss : 0.074
fin


In [ ]:
train_D.shape

In [ ]:
np.array(new_train).shape

In [ ]:
train_y.shape

In [ ]:
test_csv.shape

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(random_state = 42, C = 0.5, kernel = 'linear')
params = {
    #'decision_function_shape' : [ 'ovr', 'ova'],
    #'C' : [0.5,1,10], 'kernel':['rbf','linear']
}
cv = GridSearchCV(clf, param_grid = params)
cv.fit(np.array(new_train), train_y[:-52])



/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=0.5, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='linear', max_iter=-1,
                           probability=False, random_state=42, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None, param_grid={},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
cv.best_params_# 0.5, ovr, linear

{}

In [ ]:
with torch.no_grad():
  model.eval()
  new_test, _=  model(test_D.to(device))

In [ ]:
new_test.shape

torch.Size([2947, 192])

In [ ]:
for_test = new_test.cpu()#np.resize(new_test.cpu(), (2947, 64*3))

In [ ]:
pred = cv.predict(for_test)#(new_test.cpu().reshape(2947, 128*3))

In [ ]:
np.array(pred).shape

(2947,)

In [ ]:
test_y.shape

(2947, 1)

In [ ]:
print(classification_report(pred, test_y))

              precision    recall  f1-score   support

           1       0.96      0.85      0.90       557
           2       0.89      0.94      0.91       444
           3       0.92      0.95      0.94       405
           4       0.75      0.79      0.77       465
           5       0.83      0.82      0.82       539
           6       1.00      1.00      1.00       537

    accuracy                           0.89      2947
   macro avg       0.89      0.89      0.89      2947
weighted avg       0.89      0.89      0.89      2947



In [ ]:
acc = accuracy_score(pred, test_y)
print("CNN_SVM :",acc)

CNN_SVM : 0.8903970139124533


#LSTM + SVM

In [ ]:
class LSTM(torch.nn.Module):
  def __init__(self, n_class, bi):
    super(LSTM, self).__init__()

    self.lstm = torch.nn.LSTM(9, 32)
    #self.lstm2 = torch.nn.LSTM(32, 32)
    self.l1 = torch.nn.Linear(128*32, n_class)
    self.dropout = torch.nn.Dropout(0.1)
    self.tanh = torch.nn.Tanh()
    self.softmax = torch.nn.Softmax(dim = 1)

  def forward(self, x):
    #print(x.shape)
    #x = x.permute(1,0,2)# 축 바꾸기. 전치행렬처럼
    #print(x.shape)
    out, hidden = self.lstm(x)
    #out, _  = self.lstm2(out, hidden)
    #print(out.shape)
    out = self.tanh(out)
    #out = self.dropout(out)
    out = out.contiguous().view(-1, 128*32)
    #print(out.shape)
    model = self.l1(out)
    #out = self.softmax(out)
    return out, model
#소프트맥스 지말고 탄젠트나써라

In [ ]:
model = LSTM(train_L.shape[1], False).to(device)

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

In [ ]:
running_loss = 0
epoch = 200
new_train = []
for e in range(epoch+1):
  for i , data in enumerate(train_loader):
    x, y = data
    x = x.to(device)
    y = y.to(device)
    optimizer.zero_grad()
    new_t, h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss  += loss.item()
    if e == epoch:
      new_train.extend(new_t.cpu().detach().numpy())
  if (e) % 10 == 0:
    print('[%d] loss : %.3f' %(e, running_loss / 100))
    running_loss = 0

print('fin')

[0] loss : 0.078
[10] loss : 0.394
[20] loss : 0.211
[30] loss : 0.161
[40] loss : 0.133
[50] loss : 0.124
[60] loss : 0.114
[70] loss : 0.108
[80] loss : 0.096
[90] loss : 0.099
[100] loss : 0.089
[110] loss : 0.087
[120] loss : 0.080
[130] loss : 0.074
[140] loss : 0.078
[150] loss : 0.073
[160] loss : 0.069
[170] loss : 0.071
[180] loss : 0.068
[190] loss : 0.066
[200] loss : 0.062
fin


In [ ]:
np.array(new_train).shape

(7300, 4096)

In [ ]:
#new_train = np.resize(new_train, (7400,))

In [ ]:
new_train

[array([ 0.01777264,  0.06048835, -0.01753551, ..., -0.04283169,
         0.04891417,  0.04133761], dtype=float32),
 array([ 0.00207993,  0.0313543 , -0.00095009, ..., -0.03336694,
         0.04389487,  0.04810051], dtype=float32),
 array([-0.00654274,  0.01649543,  0.00906717, ..., -0.02660595,
         0.04047364,  0.04541314], dtype=float32),
 array([-0.01196849,  0.0085832 ,  0.01459578, ..., -0.023973  ,
         0.04107356,  0.03982279], dtype=float32),
 array([-0.01465636,  0.00432397,  0.01668845, ..., -0.02235164,
         0.04303858,  0.03312483], dtype=float32),
 array([-0.0169461 ,  0.00208852,  0.01784271, ..., -0.02072057,
         0.04537154,  0.0259371 ], dtype=float32),
 array([-0.01811997,  0.00089237,  0.01792175, ..., -0.01787638,
         0.04674875,  0.01804803], dtype=float32),
 array([-1.8954560e-02,  6.4177955e-05,  1.8106893e-02, ...,
        -1.4892922e-02,  4.7903009e-02,  1.0539680e-02], dtype=float32),
 array([-0.01878494, -0.00055484,  0.01760163, ..., -0

In [ ]:
train_y

,label
0,5
1,5
2,5
3,5
4,5
...,...
7347,2
7348,2
7349,2
7350,2


In [ ]:
PATH = './file.pt'
torch.save({
            'epoch': 200,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,

            }, PATH)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(random_state = 42, C = 0.5, kernel = 'linear')
params = {
    #'C' : [0.05, 0.5,1,10], 'kernel':['rbf','linear'], 'decision_function_shape':['ova','ovr']
}
cv = GridSearchCV(clf, param_grid = params)
cv.fit(np.array(new_train), np.array(train_y[:-52]))


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=0.5, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='linear', max_iter=-1,
                           probability=False, random_state=42, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None, param_grid={},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
with torch.no_grad():
  model.eval()
  new_test, _=  model(test_D.to(device))

In [ ]:
new_test = np.array(new_test.cpu())#np.resize(new_test, (2996,128))

In [ ]:
pred = cv.predict(new_test)

In [ ]:
print(classification_report(pred, test_y))

              precision    recall  f1-score   support

           1       0.99      0.97      0.98       509
           2       0.99      0.97      0.98       483
           3       0.92      0.98      0.95       397
           4       0.93      0.94      0.94       486
           5       0.94      0.94      0.94       535
           6       1.00      1.00      1.00       537

    accuracy                           0.96      2947
   macro avg       0.96      0.96      0.96      2947
weighted avg       0.96      0.96      0.96      2947



In [ ]:
acc = accuracy_score(pred, test_y)
print("LSTM_SVM :",acc)

LSTM_SVM : 0.9613165931455717


#bi-LSTM + SVM

In [ ]:
model = LSTM(train_L.shape[1], True).to(device)

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

In [ ]:
running_loss = 0
epoch = 200
new_train = []
for e in range(epoch+1):
  for i , data in enumerate(train_loader):
    x, y = data
    x = x.to(device)
    y = y.to(device)
    optimizer.zero_grad()
    new_t, h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss  += loss.item()
    if e == epoch:
      new_train.extend(new_t.cpu().detach().numpy())
  if (e) % 10 == 0:
    print('[%d] loss : %.3f' %(e, running_loss / 100))
    running_loss = 0

print('fin')

[0] loss : 0.081
[10] loss : 0.393
[20] loss : 0.205
[30] loss : 0.161
[40] loss : 0.139
[50] loss : 0.124
[60] loss : 0.112
[70] loss : 0.108
[80] loss : 0.096
[90] loss : 0.093
[100] loss : 0.087
[110] loss : 0.082
[120] loss : 0.079
[130] loss : 0.074
[140] loss : 0.074
[150] loss : 0.071
[160] loss : 0.066
[170] loss : 0.068
[180] loss : 0.063
[190] loss : 0.063
[200] loss : 0.063
fin


In [ ]:
np.array(new_train).shape

(7300, 4096)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(random_state = 42, C = 0.5, kernel = 'linear')
params = {
    #'C' : [0.05, 0.5,1,10], 'kernel':['rbf','linear'], 'decision_function_shape':['ova','ovr']
}
cv = GridSearchCV(clf, param_grid = params)
cv.fit(np.array(new_train), np.array(train_y[:-52]))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=0.5, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='linear', max_iter=-1,
                           probability=False, random_state=42, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None, param_grid={},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
with torch.no_grad():
  model.eval()
  new_test, _=  model(test_D.to(device))

In [ ]:
new_test = np.array(new_test.cpu())

In [ ]:
pred = cv.predict(new_test)

In [ ]:
print(classification_report(pred, test_y))

              precision    recall  f1-score   support

           1       0.93      0.95      0.94       487
           2       0.90      0.93      0.92       460
           3       0.89      0.85      0.87       438
           4       0.95      0.87      0.91       535
           5       0.87      0.94      0.90       490
           6       1.00      1.00      1.00       537

    accuracy                           0.93      2947
   macro avg       0.92      0.92      0.92      2947
weighted avg       0.93      0.93      0.92      2947



In [ ]:
acc = accuracy_score(pred, test_y)
print("bi-LSTM_SVM :",acc)

bi-LSTM_SVM : 0.9250084832032576
